In [1]:
%cd ../..
%pip install .
import time
# def placeObjs(car, numCars):
#     for i in range(numCars):
#         car = Car ahead of car by Range(4, 5)
#         leftCar = Car left of car by Normal(2, 0.1), facing roadDirection
#         rightCar = Car right of car by Normal(3, 0.1), facing Range(0, 10) deg relative to ego.heading
#     return leftCar, rightCar

# spawn_point = 207.26 @ 8.72
# ego = Car at spawn_point, with visible_distance 200

# leftCar, rightCar = placeObjs(ego, 2)
# require (distance to leftCar) < 200
# require (distance to rightCar) < 200

/Users/byronhsu/Desktop/apperception
Processing /Users/byronhsu/Desktop/apperception
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for apperception: filename=apperception-0.1.0-py3-none-any.whl size=74735 sha256=9e71e91f827c9344229d76a701bda115d8e0054498a9dd67d3e85492f53d3984
  Stored in directory: /private/var/folders/jk/04mdk5z529z3kdnl2cl6ty3h0000gn/T/pip-ephem-wheel-cache-_253itxw/wheels/73/ed/08/f70e9954595ae1250cabcf7ad1fe482879fdb2b7b75b94eef9
Successfully built apperception
  Attempting uninstal

In [2]:
"""
with (
  select *
  from item_traj
  where item_traj.object_type = 'car'
) as cars

select *
from cars as t1
join cars as t2 on t1.cameraId = t2.cameraId
join Cameras as cam on t1.cameraId = Cameras.id
where
  getX(t1.traj, cam.timestamp) < getX(cam.egoTranslation, cam.timestamp) AND
  getX(t2.traj, cam.timestamp) > getX(cam.egoTranslation, cam.timestamp) AND
  facingRelative(t1.heading, road_direction(t1.centroid, cam.timestamp), cam.timestamp) = 0 AND
  facingRelative(t2.heading, cam.egoHeading, cam.timestamp) >= 0 AND
  facingRelative(t2.heading, cam.egoHeading, cam.timestamp) < 10 AND
  DISTANCE(cam.egoTranslation, t1.centroid, cam.timestamp) < 200 AND
  DISTANCE(cam.egoTranslation, t2.centroid, cam.timestamp) < 200
"""
"""
Need to implement:
  SQL function getX(tgeompoints, timestamptz)
  SQL function getX(geometry)
  SQL function road_direction(tgeompoint, timestamptz)
  SQL function road_direction(geometry)
  SQL function facingRelative(tfloat, real, timestamptz)
  SQL function facingRelative(real, real)
  SQL function distance(geometry, tgeompoint, timestamptz)
  SQL function distance(geometry, geometry)
"""

'\nNeed to implement:\n  SQL function getX(tgeompoints, timestamptz)\n  SQL function getX(geometry)\n  SQL function road_direction(tgeompoint, timestamptz)\n  SQL function road_direction(geometry)\n  SQL function facingRelative(tfloat, real, timestamptz)\n  SQL function facingRelative(real, real)\n  SQL function distance(geometry, tgeompoint, timestamptz)\n  SQL function distance(geometry, geometry)\n'

In [3]:
from apperception.new_world import *
from apperception.new_db import database
database.reset()
from benchmarks.ingest_scenic_data import ingest_data
ingest_data()


ingest all scenic data
ingestion done


In [4]:
### Prepare the world for queries ###
from apperception.new_world import *
# directly ingest the mini dataset and boston roadnetwork if needed
# from apperception.new_db import database
# database.reset()
# from benchmarks.ingest_scenic_data import ingest_data
# ingest_data()

name = 'ScenicWorld' # world name
world = empty_world(name=name)

# ### Query ###
from apperception.utils import F
car_world = world.filter(lambda obj: obj.object_type == 'vehicle.car')

def pred_1(obj1, obj2, cam):
    return (
        F.get_x(obj1.traj, cam.timestamp) < F.get_x(cam.ego, cam.timestamp) and
        F.get_x(obj2.traj, cam.timestamp) > F.get_x(cam.ego, cam.timestamp)
    )
world_1 = car_world.filter(pred_1)

In [5]:
start = time.time()

key_1 = world_1.get_id_and_time(num_joined_tables=2)

end = time.time()
print(format(end-start))

execute: filter
execute: filter
execute: get_id_and_time
get_id_and_time 
        SELECT table_0.itemId,table_1.itemId, cameras.timestamp
        FROM (
        SELECT DISTINCT *
        FROM (SELECT * FROM item_general_trajectory) as table_0
        
        
        WHERE (table_0.objectType='vehicle.car')
        ) as table_0
        JOIN (
        SELECT DISTINCT *
        FROM (SELECT * FROM item_general_trajectory) as table_0
        
        
        WHERE (table_0.objectType='vehicle.car')
        ) as table_1 ON table_1.cameraId = table_0.cameraId
        JOIN Cameras ON Cameras.cameraId = table_0.cameraId
        WHERE ((getX(table_0.trajCentroids, Cameras.timestamp)<getX(Cameras.egoTranslation, Cameras.timestamp)) AND (getX(table_1.trajCentroids, Cameras.timestamp)>getX(Cameras.egoTranslation, Cameras.timestamp)))
        
done execute node
0.9225616455078125


In [6]:
key_1

[('296e7cae172b412a8759854d1fd8569a',
  '1ff902dcf7ed4ef1952ef98ffe54d109',
  datetime.datetime(2018, 8, 1, 12, 54, 13, 912404, tzinfo=datetime.timezone.utc)),
 ('296e7cae172b412a8759854d1fd8569a',
  '2d4eb161afa1425dac77e1cf16683287',
  datetime.datetime(2018, 8, 1, 12, 54, 13, 912404, tzinfo=datetime.timezone.utc)),
 ('296e7cae172b412a8759854d1fd8569a',
  '7c1e718215a64160994b5dd294c645d8',
  datetime.datetime(2018, 8, 1, 12, 54, 13, 912404, tzinfo=datetime.timezone.utc)),
 ('296e7cae172b412a8759854d1fd8569a',
  '94e80b2db3f740a8b2ce29b15a466ec2',
  datetime.datetime(2018, 8, 1, 12, 54, 13, 912404, tzinfo=datetime.timezone.utc)),
 ('296e7cae172b412a8759854d1fd8569a',
  'de898d510f6443bc9b22bd26d90e239b',
  datetime.datetime(2018, 8, 1, 12, 54, 13, 912404, tzinfo=datetime.timezone.utc)),
 ('296e7cae172b412a8759854d1fd8569a',
  'faf5a1c0630840339ff84fb0c8dac38a',
  datetime.datetime(2018, 8, 1, 12, 54, 13, 912404, tzinfo=datetime.timezone.utc)),
 ('3d4a9a1b28c2407b89a544511fbe1698',
  

In [7]:
def pred_2(obj1, obj2, cam):
    return (
#         F.facing_relative(obj1, F.road_direction(obj1, cam.timestamp), cam.timestamp) == 0 and
        F.facing_relative(obj2, cam.ego, cam.timestamp) >= 0 and
        F.facing_relative(obj2, cam.ego, cam.timestamp) < 10
    )

world_2 = car_world.filter(pred_2)

In [8]:
start = time.time()

key_2 = world_2.get_id_and_time(num_joined_tables=2)

end = time.time()
print(format(end-start))

execute: filter
execute: filter
execute: get_id_and_time
get_id_and_time 
        SELECT table_0.itemId,table_1.itemId, cameras.timestamp
        FROM (
        SELECT DISTINCT *
        FROM (SELECT * FROM item_general_trajectory) as table_0
        
        
        WHERE (table_0.objectType='vehicle.car')
        ) as table_0
        JOIN (
        SELECT DISTINCT *
        FROM (SELECT * FROM item_general_trajectory) as table_0
        
        
        WHERE (table_0.objectType='vehicle.car')
        ) as table_1 ON table_1.cameraId = table_0.cameraId
        JOIN Cameras ON Cameras.cameraId = table_0.cameraId
        WHERE ((facingRelative(table_1.itemHeadings, Cameras.egoHeading, Cameras.timestamp)>=0) AND (facingRelative(table_1.itemHeadings, Cameras.egoHeading, Cameras.timestamp)<10))
        
done execute node
0.310269832611084


In [9]:
key_2

[('0e5684ceded44cb190e83e45d8700d7f',
  '6abeb111755e457ebccb263fe0da1569',
  datetime.datetime(2018, 8, 1, 12, 54, 35, 262404, tzinfo=datetime.timezone.utc)),
 ('07e961904d03490ca552711c1b7c8bcf',
  '6abeb111755e457ebccb263fe0da1569',
  datetime.datetime(2018, 8, 1, 12, 54, 35, 262404, tzinfo=datetime.timezone.utc)),
 ('3dfcaa3c0092413eb5038d8eacf1fb5c',
  '6abeb111755e457ebccb263fe0da1569',
  datetime.datetime(2018, 8, 1, 12, 54, 35, 262404, tzinfo=datetime.timezone.utc)),
 ('2f19978be3384b958c9bcdc6e676895a',
  '6abeb111755e457ebccb263fe0da1569',
  datetime.datetime(2018, 8, 1, 12, 54, 35, 262404, tzinfo=datetime.timezone.utc)),
 ('63e7787ed32d4af6b48a01f4ab318d29',
  '6abeb111755e457ebccb263fe0da1569',
  datetime.datetime(2018, 8, 1, 12, 54, 35, 262404, tzinfo=datetime.timezone.utc)),
 ('5f27a9c508954a36a3d3ee5606b16bf0',
  '6abeb111755e457ebccb263fe0da1569',
  datetime.datetime(2018, 8, 1, 12, 54, 35, 262404, tzinfo=datetime.timezone.utc)),
 ('6abeb111755e457ebccb263fe0da1569',
  

In [10]:
def pred_3(obj1, obj2, cam):
    return (
        F.distance(cam.ego, obj1, cam.timestamp) < 200 and
        F.distance(cam.ego, obj2, cam.timestamp) < 200
    )
world_3 = car_world.filter(pred_3)

In [11]:
start = time.time()

key_3 = world_3.get_id_and_time(num_joined_tables=2)

end = time.time()
print(format(end-start))

execute: filter
execute: filter
execute: get_id_and_time
get_id_and_time 
        SELECT table_0.itemId,table_1.itemId, cameras.timestamp
        FROM (
        SELECT DISTINCT *
        FROM (SELECT * FROM item_general_trajectory) as table_0
        
        
        WHERE (table_0.objectType='vehicle.car')
        ) as table_0
        JOIN (
        SELECT DISTINCT *
        FROM (SELECT * FROM item_general_trajectory) as table_0
        
        
        WHERE (table_0.objectType='vehicle.car')
        ) as table_1 ON table_1.cameraId = table_0.cameraId
        JOIN Cameras ON Cameras.cameraId = table_0.cameraId
        WHERE ((distance(Cameras.egoTranslation, table_0.trajCentroids, Cameras.timestamp)<200) AND (distance(Cameras.egoTranslation, table_1.trajCentroids, Cameras.timestamp)<200))
        
done execute node
1.8464388847351074


In [12]:
key_3

[('1ff902dcf7ed4ef1952ef98ffe54d109',
  '1ff902dcf7ed4ef1952ef98ffe54d109',
  datetime.datetime(2018, 8, 1, 12, 54, 13, 912404, tzinfo=datetime.timezone.utc)),
 ('1ff902dcf7ed4ef1952ef98ffe54d109',
  '296e7cae172b412a8759854d1fd8569a',
  datetime.datetime(2018, 8, 1, 12, 54, 13, 912404, tzinfo=datetime.timezone.utc)),
 ('1ff902dcf7ed4ef1952ef98ffe54d109',
  '2d4eb161afa1425dac77e1cf16683287',
  datetime.datetime(2018, 8, 1, 12, 54, 13, 912404, tzinfo=datetime.timezone.utc)),
 ('1ff902dcf7ed4ef1952ef98ffe54d109',
  '3d4a9a1b28c2407b89a544511fbe1698',
  datetime.datetime(2018, 8, 1, 12, 54, 13, 912404, tzinfo=datetime.timezone.utc)),
 ('1ff902dcf7ed4ef1952ef98ffe54d109',
  '7c1e718215a64160994b5dd294c645d8',
  datetime.datetime(2018, 8, 1, 12, 54, 13, 912404, tzinfo=datetime.timezone.utc)),
 ('1ff902dcf7ed4ef1952ef98ffe54d109',
  '94e80b2db3f740a8b2ce29b15a466ec2',
  datetime.datetime(2018, 8, 1, 12, 54, 13, 912404, tzinfo=datetime.timezone.utc)),
 ('1ff902dcf7ed4ef1952ef98ffe54d109',
  

In [13]:
def pred(obj1, obj2, cam):
    return (
        F.get_x(obj1.traj, cam.timestamp) < F.get_x(cam.ego, cam.timestamp) and
        F.get_x(obj2.traj, cam.timestamp) > F.get_x(cam.ego, cam.timestamp) and
#         F.facing_relative(obj1, F.road_direction(obj1, cam.timestamp), cam.timestamp) == 0 and
        F.facing_relative(obj2, cam.ego, cam.timestamp) >= 0 and
        F.facing_relative(obj2, cam.ego, cam.timestamp) < 10 and
        F.distance(cam.ego, obj1, cam.timestamp) < 200 and
        F.distance(cam.ego, obj2, cam.timestamp) < 200
    )
final_world = car_world.filter(pred)
start = time.time()
keys = final_world.get_id_and_time(num_joined_tables=2)
end = time.time()
print(format(end-start))

execute: filter
execute: filter
execute: get_id_and_time
get_id_and_time 
        SELECT table_0.itemId,table_1.itemId, cameras.timestamp
        FROM (
        SELECT DISTINCT *
        FROM (SELECT * FROM item_general_trajectory) as table_0
        
        
        WHERE (table_0.objectType='vehicle.car')
        ) as table_0
        JOIN (
        SELECT DISTINCT *
        FROM (SELECT * FROM item_general_trajectory) as table_0
        
        
        WHERE (table_0.objectType='vehicle.car')
        ) as table_1 ON table_1.cameraId = table_0.cameraId
        JOIN Cameras ON Cameras.cameraId = table_0.cameraId
        WHERE ((getX(table_0.trajCentroids, Cameras.timestamp)<getX(Cameras.egoTranslation, Cameras.timestamp)) AND (getX(table_1.trajCentroids, Cameras.timestamp)>getX(Cameras.egoTranslation, Cameras.timestamp)) AND (facingRelative(table_1.itemHeadings, Cameras.egoHeading, Cameras.timestamp)>=0) AND (facingRelative(table_1.itemHeadings, Cameras.egoHeading, Cameras.timestamp

In [14]:
keys

[('5f27a9c508954a36a3d3ee5606b16bf0',
  '6abeb111755e457ebccb263fe0da1569',
  datetime.datetime(2018, 8, 1, 12, 54, 35, 262404, tzinfo=datetime.timezone.utc)),
 ('d8a323f9e69a45bba45e547989d2e35e',
  '6abeb111755e457ebccb263fe0da1569',
  datetime.datetime(2018, 8, 1, 12, 54, 35, 262404, tzinfo=datetime.timezone.utc)),
 ('07e961904d03490ca552711c1b7c8bcf',
  '6abeb111755e457ebccb263fe0da1569',
  datetime.datetime(2018, 8, 1, 12, 54, 38, 262404, tzinfo=datetime.timezone.utc)),
 ('07e961904d03490ca552711c1b7c8bcf',
  'd8a323f9e69a45bba45e547989d2e35e',
  datetime.datetime(2018, 8, 1, 12, 54, 38, 262404, tzinfo=datetime.timezone.utc)),
 ('0e5684ceded44cb190e83e45d8700d7f',
  '6abeb111755e457ebccb263fe0da1569',
  datetime.datetime(2018, 8, 1, 12, 54, 38, 262404, tzinfo=datetime.timezone.utc)),
 ('0e5684ceded44cb190e83e45d8700d7f',
  'd8a323f9e69a45bba45e547989d2e35e',
  datetime.datetime(2018, 8, 1, 12, 54, 38, 262404, tzinfo=datetime.timezone.utc)),
 ('3dfcaa3c0092413eb5038d8eacf1fb5c',
  

In [15]:
filenames = world.get_filenames_by_itemids_and_timestamps([keys[0][0]], [keys[0][2]])

[!] This is a quick hack!

                SELECT filename FROM cameras
                WHERE cameraid IN (
                    SELECT cameraid FROM item_general_trajectory
                    WHERE itemid = '5f27a9c508954a36a3d3ee5606b16bf0'
                ) AND timestamp = timestamptz '2018-08-01 12:54:35.262404+00:00'
            
